# Preparation scripts
this notebook prepares posebusters-related files to be in the format the lddt script accepts

In [7]:
import os
pdb2proteinpath = {}
pdb2ligandpath = {}
mypath = "posebusters/posebusters_benchmark_set"
for path, dirs, files in os.walk(mypath):
    for name in (files + dirs):
        if "pdb" in name:
            pdbid = name.split("_")[0]
            pdb2proteinpath[pdbid] = f"/home/{os.path.join(path, name)}"
            ligand_name = "_".join(name.split("_")[:2])+"_ligand.sdf"
            pdb2ligandpath[pdbid] = f"/home/{os.path.join(path, ligand_name)}"
        

In [13]:
import os
from pathlib import Path
from rdkit import Chem
mypath = "pb2"
Path("pb_proc").mkdir(parents=True, exist_ok=True)
lddt_input_val = []
for path, dirs, files in os.walk(mypath):
    for f in files:
        if "json" in f:
            pb_name=f.split("_")[0]
    for name in (files + dirs):
        if "top3_poses.sdf" in name:
            sdfpath = os.path.join(path, name)
            try:
                mol = [m for m in Chem.SDMolSupplier(sdfpath) if m.GetProp("metric")=="combo"][0]
                sdfloc = f"pb_proc/{pb_name}.sdf"
                wri = Chem.SDWriter(sdfloc)
                wri.write(mol)
                wri.close()
                apoloc = pdb2proteinpath[pb_name]
                ligloc = pdb2ligandpath[pb_name]
                lddt_input_val.append((apoloc,ligloc,f"/home/{sdfloc}"))
            except Exception as e:
                print("problem with ",name[-19:-15],e,name)

import csv
with open('input_pb.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL,lineterminator='\n')
    writer.writerow(["apo","ref","lig"])
    for line in lddt_input_val:
        writer.writerow(line)